# Dataset repacking implementation

Advance dataset operations, of sorting, offset, and length support

In [2]:
GPU_DEVICES="auto"
ENABLE_WANDB=True
WANDB_PREFIX="infctx-v5-datapack"

print("ENABLE_WANDB:", ENABLE_WANDB)
print("GPU_DEVICES:", GPU_DEVICES)

if ENABLE_WANDB:
    WANDB_MODE="online"
else:
    WANDB_MODE="disabled"

# Computing the notebook, and various paths
import os
NOTEBOOK_DIR=os.path.dirname(os.path.abspath("__file__"))
PROJECT_DIR=os.path.abspath(os.path.join(NOTEBOOK_DIR, "../../"))
TRAINER_DIR=os.path.abspath(os.path.join(PROJECT_DIR, "./RWKV-v5/"))

print("NOTEBOOK_DIR:", NOTEBOOK_DIR)
print("TRAINER_DIR:", TRAINER_DIR)
print("PROJECT_DIR:", PROJECT_DIR)

ENABLE_WANDB: True
GPU_DEVICES: auto
NOTEBOOK_DIR: /workspace/picocreator/RWKV-infctx-trainer/notebook/trainer-v5-validation
TRAINER_DIR: /workspace/picocreator/RWKV-infctx-trainer/RWKV-v5
PROJECT_DIR: /workspace/picocreator/RWKV-infctx-trainer


In [2]:
# Init the model
!cd "{TRAINER_DIR}" && \
    python3 ./init_model.py \
        --n_layer 6 --n_embd 1024 \
        --vocab_size world --skip-if-exists \
        "../model/L6-D1024-world-v5base-init.pth"

[2024-02-04 14:06:15,966] [INFO] [real_accelerator.py:161:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[RWKV.model] Running RWKV infctx using 'torch-jit' with torch '2.1.2'
---- Initializing model ----
No of layers: 6
Embedding size: 1024
Output model path: ../model/L6-D1024-world-v5base-init.pth
Vocab size: 65536
Emb scale: 0.0001
Note: this process takes a significant time (and ram) for large models
---- ----- ----
Model exists, skipping init_model


# Build the datapack

In [6]:
# Lets preload the requried dataset 
!cd "{TRAINER_DIR}" && \
    python3 datapack_build.py "{NOTEBOOK_DIR}/config/datapack-build.yaml"

>> Starting datapack build process for: /workspace/picocreator/RWKV-infctx-trainer/notebook/trainer-v5-validation/config/datapack-build.yaml
>> Preparing dataset - index:  0
Saving the dataset (1/1 shards): 100%|█| 586/586 [00:00<00:00, 4641.04 examples/
Saving the dataset (1/1 shards): 100%|████| 6/6 [00:00<00:00, 270.57 examples/s]
>> Preparing dataset - index:  1
Map (num_proc=160): 100%|████| 240402/240402 [00:02<00:00, 113556.24 examples/s]
Saving the dataset (1/1 shards): 100%|█| 9232/9232 [00:01<00:00, 8685.37 example
Saving the dataset (1/1 shards): 100%|█| 2429/2429 [00:00<00:00, 40807.03 exampl
>> Dataset Mixing mode:  shuffle
>> Saving dataset to data_path :  ../datapath/v5-validation/example-datapack/
Saving the dataset (1/1 shards): 100%|█| 9818/9818 [00:00<00:00, 12069.53 exampl
Saving the dataset (1/1 shards): 100%|█| 2435/2435 [00:00<00:00, 38873.99 exampl
>> Dataset saved to data_path
>> -----------------------------------
>> Performing dataset counting
>> ------------

In [7]:
# Lets load the datapath at {PROJECT_DIR}/datapath/world/Eagle-x-multipack/ via HF dataset
# and iterate the first 10 documents
import datasets
datapath = f"{PROJECT_DIR}/datapath/v5-validation/example-datapack/"
print(f"Loading the dataset... {datapath}")

# Load the dataset
full_dataset = datasets.load_from_disk(datapath)
print(f"Dataset loaded, {len(full_dataset)} documents")

# Train dataset
train_dataset = full_dataset["train"]

# Lets iterate the first 10 documents
iterate_limit = 10
for idx, doc in enumerate(train_dataset):
    print(f"Document {idx+1}:")
    # Print the keys
    print("- Keys:", doc.keys())
    print("- input_ids length:", len(doc["input_ids"]))
    print("- attention_mask length:", len(doc["attention_mask"]))
    print("- token_type_ids length:", len(doc["token_type_ids"]))
    # print("- sample_length:", doc["sample_length"])

    if idx >= iterate_limit:
        break

Loading the dataset... /workspace/picocreator/RWKV-infctx-trainer/datapath/v5-validation/example-datapack/


NameError: name 'dataset' is not defined

# Short train

In [ ]:
!cd "{TRAINER_DIR}" && \
    export WANDB_MODE="disabled" && \
    python3 lightning_trainer.py fit \
        -c "{NOTEBOOK_DIR}/config/datapack-train.yaml" \
        --model.load_model="../model/L6-D1024-world-v5base-init.pth" \
        --trainer.callbacks.init_args.dirpath="../checkpoint/datapack-validaiton-train/" \
        --trainer.logger.init_args.name="{WANDB_PREFIX} - Multi Datapack Validation - (deepspeed_stage_1)" \
        --trainer.strategy="deepspeed_stage_1" \
        --trainer.microbatch_size=8 \
        --trainer.fast_dev_run=2 \
        --trainer.devices="{GPU_DEVICES}"

# Partial training run

In [110]:
!cd "{TRAINER_DIR}" && \
    export WANDB_MODE="{WANDB_MODE}" && \
    python3 lightning_trainer.py fit \
        -c "{NOTEBOOK_DIR}/config/datapack-train.yaml" \
        --model.load_model="../model/L6-D1024-world-v5base-init.pth" \
        --trainer.callbacks.init_args.dirpath="../checkpoint/datapack-validaiton-train/" \
        --trainer.logger.init_args.name="{WANDB_PREFIX} - Multi Datapack Validation - (deepspeed_stage_1)" \
        --trainer.strategy="deepspeed_stage_1" \
        --trainer.microbatch_size=8 \
        --trainer.max_steps=50 \
        --trainer.devices="{GPU_DEVICES}"

[2024-01-28 09:05:15,732] [INFO] [real_accelerator.py:161:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[RWKV.model] Running RWKV infctx using 'torch-jit' with torch '2.1.2'
/home/recursal/miniconda3/envs/rwkv-infctx/lib/python3.11/site-packages/lightning/pytorch/cli.py:518: LightningCLI's args parameter is intended to run from within Python like if it were from the command line. To prevent mistakes it is not recommended to provide both args and command line arguments, got: sys.argv[1:]=['fit', '-c', '/home/recursal/RWKV-infctx-trainer/notebook/trainer-v5-validation/config/datapack-train.yaml', '--model.load_model=../model/L6-D1024-world-v5base-init.pth', '--trainer.callbacks.init_args.dirpath=../checkpoint/datapack-validaiton-train/', '--trainer.logger.init_args.name=infctx-v5-datapack - Multi Datapack Validation - (deepspeed_stage_1)', '--trainer.strategy=deepspeed_stage_1', '--trainer.microbatch_size=8', '--trainer.max_steps=50', '--trainer.devices=auto'], args=['fit